In [16]:
import numpy as np
import pandas as pd
import os
from roboflow import Roboflow

import fiftyone as fo
import fiftyone.zoo as foz
import pandas.api.types as pdt
from fiftyone import ViewField as F

rf = Roboflow(api_key=os.environ.get("ROBOFLOW_API_KEY"))

# Get Data

In [11]:
project = rf.workspace("suas-gyf9o").project("test-t0cxb")
version = project.version(2)
dataset = version.download(
    "coco", "/home/cole/cole_scripts/vision/SAETA/datasets/drone"
)
project = rf.workspace("suas-gyf9o").project("tent-xwofm-2hmcv")
version = project.version(2)
dataset = version.download("coco", "/home/cole/cole_scripts/vision/SAETA/datasets/tent")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /home/cole/cole_scripts/vision/SAETA/datasets/drone in coco:: 100%|██████████| 8439/8439 [00:09<00:00, 858.17it/s] 


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /home/cole/cole_scripts/vision/SAETA/datasets/tent in coco:: 100%|██████████| 368/368 [00:00<00:00, 1070.27it/s]


In [26]:
if "2026_SUAS" in fo.list_datasets():
    dataset = fo.load_dataset("2026_SUAS")
else:
    from convert_visdrone_to_yolo import load_datasets_from_dirs

    dataset = load_datasets_from_dirs("2026_SUAS", ["datasets/drone", "datasets/tent"])
print(dataset)

Name:        2026_SUAS
Media type:  image
Num samples: 8791
Persistent:  False
Tags:        []
Sample fields:
    id:                 fiftyone.core.fields.ObjectIdField
    filepath:           fiftyone.core.fields.StringField
    tags:               fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:         fiftyone.core.fields.DateTimeField
    last_modified_at:   fiftyone.core.fields.DateTimeField
    detections:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    person_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


# Clean Data

In [27]:
# Remove images without anotations
dataset = dataset.match(fo.ViewField("detections.detections").length() > 0)
classes = dataset.distinct("detections.detections.label")
print(f"Classes: {classes}")
print(dataset)

Classes: ['0', '1', '2', '4', 'car', 'person', 'tent']
Dataset:     2026_SUAS
Media type:  image
Num samples: 8766
Sample fields:
    id:                 fiftyone.core.fields.ObjectIdField
    filepath:           fiftyone.core.fields.StringField
    tags:               fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:         fiftyone.core.fields.DateTimeField
    last_modified_at:   fiftyone.core.fields.DateTimeField
    detections:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    person_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Match(filter={'$expr': {'$gt': [...]}})


In [28]:
# Load YOLO11m from FiftyOne zoo
model = foz.load_zoo_model("yolo11m-coco-torch")

# Filter for samples that contain tents
samples_with_tents = dataset.filter_labels("detections", F("label") == "tent").exists(
    "detections"
)

# Apply model ONLY to images with tents (this modifies the original dataset)
samples_with_tents.apply_model(model, label_field="person_predictions")

# Filter the person_predictions field to keep only "person" detections
# This only affects samples that have person_predictions (i.e., images with tents)
dataset = dataset.filter_labels(
    "person_predictions", F("label") == "person", only_matches=False
)

# dataset now contains ALL images:
# - Images with tents will have "person_predictions" field
# - Images without tents will NOT have "person_predictions" field
print(f"Total samples: {len(dataset)}")
print(f"Samples with person predictions: {len(dataset.exists('person_predictions'))}")

 100% |█████████████████| 335/335 [14.0s elapsed, 0s remaining, 21.1 samples/s]      
Total samples: 8766
Samples with person predictions: 8766


In [ ]:
# from ultralytics import YOLO
# from PIL import Image
# import fiftyone as fo
# from fiftyone import ViewField as F

# model = YOLO('yolov11s.pt')
# print("YOLO model ready")

# # Filter for samples that contain 'tent' in their ground truth labels
# samples_with_tents = dataset.filter_labels(
#     "detections",  # Your label field name
#     F("label") == "tent"
# ).exists("detections")

# print(f"Found {len(samples_with_tents)} samples with tents")

# # Add person predictions to samples with tents
# with fo.ProgressBar() as pb:
#     for sample in pb(samples_with_tents):
#         # Run YOLO inference
#         results = model(sample.filepath, verbose=False)[0]

#         # Get image dimensions
#         h, w = results.orig_shape

#         # Filter for only person detections (COCO class 0 in YOLO)
#         person_detections = []
#         for box in results.boxes:
#             class_id = int(box.cls[0])
#             if class_id == 0:  # Person class in COCO
#                 # Get box coordinates (x1, y1, x2, y2)
#                 x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
#                 confidence = float(box.conf[0])

#                 # Convert to relative coordinates [x, y, width, height]
#                 rel_box = [
#                     x1 / w,
#                     y1 / h,
#                     (x2 - x1) / w,
#                     (y2 - y1) / h
#                 ]

#                 person_detections.append(
#                     fo.Detection(
#                         label="person",
#                         bounding_box=rel_box,
#                         confidence=confidence
#                     )
#                 )

#         # Save predictions
#         sample["person_predictions"] = fo.Detections(detections=person_detections)
#         sample.save()

# print(f"Added person predictions to {len(samples_with_tents)} samples")

In [30]:
dataset = dataset.map_labels("detections", {"0": "person"})
dataset = dataset.filter_labels("detections", F("label").is_in(["tent", "person"]))

In [33]:
print(dataset)

Dataset:     2026_SUAS
Media type:  image
Num samples: 7647
Sample fields:
    id:                 fiftyone.core.fields.ObjectIdField
    filepath:           fiftyone.core.fields.StringField
    tags:               fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:         fiftyone.core.fields.DateTimeField
    last_modified_at:   fiftyone.core.fields.DateTimeField
    detections:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    person_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Match(filter={'$expr': {'$gt': [...]}})
    2. FilterLabels(field='person_predictions', filter={'$eq': ['$$this.label', 'person']}, only_matches=False, trajectories=False)
    3. MapLabels(field='detections', map={'0': 'person'})
    4. FilterLabels(field='detections', filter=

In [32]:
session = fo.launch_app(dataset)

# Export dataset

In [34]:
# The directory to which to write the exported dataset
export_dir = "/home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5"
label_field = "detections"
dataset_type = fo.types.YOLOv5Dataset
dataset.export(
    export_dir=export_dir,
    dataset_type=dataset_type,
    label_field=label_field,
)

 100% |███████████████| 7647/7647 [1.3m elapsed, 0s remaining, 410.8 samples/s]      


In [35]:
workspace = rf.workspace("suas-gyf9o")

workspace.upload_dataset(
    "/home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5",
    "2026-SUAS-v5",
    num_workers=10,
    project_license="MIT",
    project_type="object-detection",
)

loading Roboflow workspace...
loading Roboflow project...
Created project suas-gyf9o/2026-suas-v5
[UPLOADED] /home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5/images/val/0000001_03999_d_0000007_jpg.rf.5d12dfe376b3a9a5fb974e92a4c6926f.jpg (JzkTK3XTNftCalHRAOVm) [2.0s] / annotations = OK [1.0s]
[UPLOADED] /home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5/images/val/0000001_05999_d_0000011_jpg.rf.b37506764044c661ac5aa988e39a42bb.jpg (coRbXACjqwJ3pzVuXJKb) [1.9s] / annotations = OK [1.2s]
[UPLOADED] /home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5/images/val/0000001_05499_d_0000010_jpg.rf.081864b9780e4c911721ee223c8bdf84.jpg (wtxQNUDxxQxJWBo9WDA9) [2.3s] / annotations = OK [1.0s]
[UPLOADED] /home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5/images/val/0000001_02999_d_0000005_jpg.rf.843324d95cc9f936c061cafab5756698.jpg (V5BPVaOX4Ed93doGQUQg) [2.1s] / annotations = OK [1.3s]
[UPLOADED] /home/cole/cole_scripts/vision/SAETA/datasets/2026_suas_v5/images/val/-